# Improve Metadata Search

In [ ]:
# TODO: add more descriptions

Our RAG is not very good on answering questions regarding movies metadata such as release year or gnre o. 
For example, how our system deal with questions like: 
- Could you suggest a romantic movies from '90s
- I love action movies could you suggest one? 

Right now, with just the simple semantic search between query and movies plots we don't leverage the metadata stored in our database.

In [ ]:
import importlib

if not importlib.util.find_spec("ace"):
    !pip install -qqq git+https://github.com/xtreamsrl/ace-of-splades

## Let's Play with LanceDB

In [ ]:
from ace_of_splades.data import get_movies_dataset
from sentence_transformers import SentenceTransformer
from ace_of_splades.data import get_movies_dataset
import lancedb
import openai
import polars as pl
from pydantic import BaseModel

In [ ]:
data_path = f"../data"
encoder = SentenceTransformer("all-MiniLM-L6-v2")
movies = get_movies_dataset(local=True)

In [ ]:
uri = f"{data_path}/movies_embeddings"
db = lancedb.connect(uri)

movies_table = db.create_table("movies", movies, mode="overwrite")

Let's see what movies are retrieved from database just using semantic search.

In [ ]:
question = "I love '90 fantasy movies with dragons"

In [ ]:
movies_table.search(
    encoder.encode(question)
).limit(10).select(
    ['title', 'release_year', 'genre']
).to_list()

The retrieved movies looks good, they are all aventure or fantasy movies with some relation with dragosn as asked by the user. **The problem is that many of them are not from 90s!** This is due the fact that the semantic search using enbeddings doesn't taken into account the movies release year.

Fortunatelly, many vector database have a useful filtering query system. LanceDB, for example, support filtering data using a SQL-like language.

In [ ]:
movies_table.search(
    encoder.encode(question)
).where(
    "genre LIKE '%fantasy%' AND (release_year >= 1990 AND release_year < 2000)"
).select(['title', 'release_year', 'genre']).limit(5).to_list()

Hurray! Looks better, all movies now are from 90s. Now the metadata informations in the questions are useless we could just search for "dragons" and see what happen!

In [ ]:
movies_table.search(
    encoder.encode("dragons")
).where(
    "genre LIKE '%fantasy%' AND (release_year >= 1990 AND release_year < 2000)"
).select(['title', 'release_year', 'genre']).limit(5).to_list()

Even better! So, based on this experiments we need something to convert the natural language user query to a SQL-like filter (or other languages supported by your vector database). 

To do this LLM could help us. We can ask a model to do this job for us and adding a step to our RAG.

In [ ]:
QUERY_METADATA_SYS_MESSAGE = """Your goal is to structure the user's query to match the request schema provided below.
When responding, use a data structure with the following schema:

query (string): text string to compare to document contents
filter: (string): logical condition statement for filtering documents

You need to build filters for a database with the following fields: 

{fields}

The query string should contain only text expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query.
You can use standard SQL expressions as predicates for filtering operations. You can use the following list of SQL expressions:

{supported_operations}

For genre, director, cast, origin use always LIKE with % wildcard at the beggin. For example origin LIKE '%British%'

Make sure you only use the comparators and logical operators listed above and not others.
Make sure that filters only refer to attributes in the data source.
Make sure that filters only use the attributed names with their function names if there are functions applied to them.
Make sure that filters only use the format `YYYY` when handling years.
Make sure that filters take into account the descriptions of attributes and only make comparisons that are feasible given the type of data being stored.
Make sure that filters are only used as needed. If no filters should be applied, return "NO_FILTER" for the filter value.

<< Example 1. >>
User Query:
I love action American movies with superheroes

Structured Request:
"query": "Superheroes",
"filter": "genre LIKE '%action%' AND origin = 'American'"

<< Example 2. >>
User Query:
I love '90 fantasy movies with dragons

Structured Request:
"query": "Dragons",
"filter": "genre LIKE '%fantasy%' AND (release_year >= 1990 AND release_year < 2000)"
"""

supported_operations = [
    ">, >=, <, <=, =", 
    "AND, OR, NOT",
    "IS NULL, IS NOT NULL",
    "IS TRUE, IS NOT TRUE, IS FALSE, IS NOT FALSE",
    "IN",
    "LIKE, NOT LIKE",
    "CAST",
    "regexp_match(column, pattern)"
]


formatted_supported_operations = '\n'.join(supported_operations)
query_metadata_system_message = QUERY_METADATA_SYS_MESSAGE.format(fields =movies_table.schema, supported_operations = formatted_supported_operations)

In [ ]:
print(query_metadata_system_message)

In [ ]:
class VectorDBQuery(BaseModel):
    query: str
    filter: str

In [ ]:
def vectordb_query_builder(user_question: str, system_message: str = query_metadata_system_message):
    client = openai.OpenAI()
    prompt = f"Could you generate query and filter for the following user natural language question: {user_question}"
    
    chat_completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
        response_format=VectorDBQuery
    )
    
    return chat_completion.choices[0].message.parsed

In [ ]:
def get_records(
        query: VectorDBQuery, *, encoder=encoder, db_table=movies_table, max_results=10, verbose = False
):
    query_vector = encoder.encode(query.query).tolist()
    columns = ['release_year', 'title', 'origin', 'director', 'cast', 'genre', 'plot', '_distance']

    if verbose: 
        print(f"Query: {query.query}")
        print(f"Filter: {query.filter}")
    
    if query.filter == 'NO_FILTER':
        return db_table.search(
            query_vector
        ).limit(max_results).select(
            columns
        ).to_list()
    else: 
        return db_table.search(
            query_vector
        ).where(query.filter).limit(max_results).select(
            columns
        ).to_list()

Now that we have improve the `get_records` methods taking into consideration also metadata. We could build the rag again! 

In [ ]:
SYSTEM_MESSAGE = """ You are a movie expert whose goal is to recommend a good movie to the user.

RULES: 
- You should reply to questions about movie plots or Synopsys, movies metadata (release date, cast, or director), and provide plot summary;
- For every question outside the scope, please reply politely that you're not able to provide a response and describe briefly your scope;
- Don't mention that you have a list of films as a context. This should be transparent to the user
- If you don't have the movie in your context, reply that you don't know how to reply"""

In [ ]:
prompt_template = """
  Here are some suggested movies (ranked by relevance) to help you with your choice.
  {context}

  Use these suggestions to answer this question:
  {question}
"""

context_template = """
Title: {title}
Release date: {release_year}
Director: {director}
Cast: {cast}
Genre: {genre}
Overview: {plot}
"""


def format_records_into_context(records, *, template):
    return "".join(
        context_template.format(
            title=rec["title"],
            release_year=rec["release_year"],
            director=rec["director"],
            cast=rec["cast"],
            genre=rec["genre"],
            plot=rec["plot"],
        )
        for rec in records
    )

In [ ]:
client = openai.OpenAI()


def ask(
        question,
        *,
        max_results=10,
        system=SYSTEM_MESSAGE,
        prompt_template=prompt_template,
        context_template=context_template,
        db_table=movies_table,
        verbose=False
):
    db_query = vectordb_query_builder(question)
    records = get_records(
        query=db_query, max_results=max_results, db_table=movies_table, verbose = verbose
    )

    context = format_records_into_context(records, template=context_template)

    prompt = prompt_template.format(question=question, context=context)

    chat_completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ],
    )

    answer = chat_completion
    if verbose:
        print(answer.choices[0].message.content)

    return answer

# Results

In [ ]:
question = "I love '90 fantasy movies with dragons"
answer = ask(question=question, verbose=True)
print()
print(answer.choices[0].message.content)

In [ ]:
question = "How many Rocky movies where filmed?"
answer = ask(question=question, verbose=True) 
print()
print(answer.choices[0].message.content)

In [ ]:
question = "I love Turkish movies, and my prefered director is Özpetek! What movie I can see?"
answer = ask(question=question, verbose=True) 
print(answer.choices[0].message.content)